In [24]:
import pandas as pd

import os


path = "./dll_analysis/malware"
filename = "fixed_rules.csv"
file_to_scan = os.path.join(path,filename)
df1 = pd.read_csv(file_to_scan)

temp_families = open(path+"/most_frequent_families.csv", "r").readlines()

In [26]:
print(temp_families)
most_frequent_families = set()
"""NOTE: check the format when moving to EXE. This was for DLL only."""
for i in range(len(temp_families)):
    #if(i==0):
        #continue
    most_frequent_families.add(temp_families[i].strip('\n').split(",")[1])
""" For EXEs, or, in general, for any file as a list of families one per line:

for i in range(len(temp_families)):
    most_frequent_families.append(temp_families[i].strip('\n'))
"""
print(most_frequent_families)

['1,epack\n', '2,locky\n', '3,razy\n', '4,zusy\n', '6,agentb\n', '7,locky\n', '8,midie\n', '9,ranserkd\n', '10,razy\n', '11,waldek\n', '12,locky\n', '13,locky\n']
set(['razy', 'locky', 'agentb', 'midie', 'epack', 'ranserkd', 'waldek', 'zusy'])


In [27]:
import pefile
import ssdeep

m1 = df1.loc[:,["Exe name", "Family", "first_seen", "ssdeep"]]
m1["first_seen:month"] = pd.to_datetime(m1["first_seen"]).dt.to_period("M")

In [28]:
del m1["first_seen"]

In [29]:
m1.head()
m1 = m1.dropna()

In [30]:
df_dict={el: pd.DataFrame() for el in most_frequent_families}

In [31]:
import ssdeep
for family in most_frequent_families:
    print(family)
    mf = m1[:][m1.Family == family]
    most_frequent_months = mf.groupby(mf["first_seen:month"]).size().reset_index(name="count")
    
    most_frequent_months = most_frequent_months[most_frequent_months["count"]>5]["first_seen:month"]
    df_month ={ el: pd.DataFrame() for el in most_frequent_months.values}
    for month in most_frequent_months.values:
        mm = mf[:][mf["first_seen:month"]==month]
        print(mm.shape)
        cp = mm.merge(mm, how = 'outer', on = "Family")
        """ Performing cartesian product of the dataset with itself
    ( it will be useful to use the apply function over the same row of the dataframe).
    Each row will now contain two samples of the same family 
    (too complex and, probably, useless to perform it over all the samples, independently on the family)"""
        cp["Similarity"] = cp.apply(lambda x: ssdeep.compare(str(x["ssdeep_x"]),str(x["ssdeep_y"])), axis=1)
        df_month[month] = cp
        #print(cp)
    df_dict[family] = df_month


razy
(25, 4)
(6, 4)
locky
(275, 4)
(653, 4)
agentb
midie
epack
ranserkd
(12, 4)
waldek
zusy


In [32]:
import os
os.mkdir(path+"/heatmap/ssdeep/monthly_analysis")

In [33]:
import matplotlib.pyplot as plt
import gc

for family in most_frequent_families:
    os.mkdir(path+"/heatmap/ssdeep/monthly_analysis/"+family)
    for month, df_month in df_dict[family].items():
        print(family, month)
        df_month = df_month.pivot(index= "Exe name_x", columns="Exe name_y", values="Similarity")
        ax = plt.pcolor(df_month)
        ax.get_figure().savefig(path+"/heatmap/ssdeep/monthly_analysis/"+family+"/"+str(month)+"_heatmap", dpi=600)
        plt.clf()
        del ax
        gc.collect()
        df_month.to_csv(path+"/heatmap/ssdeep/monthly_analysis/"+family+"/"+str(month)+"_heatmap.csv")

('razy', Period('2016-11', 'M'))
('razy', Period('2016-10', 'M'))
('locky', Period('2016-11', 'M'))
('locky', Period('2016-10', 'M'))
('ranserkd', Period('2016-11', 'M'))


In [23]:
print(path)

./exe_analysis/malware
